In [14]:
import pandas as pd
import re
import json

In [35]:
with open('/Users/bilindhajer/BilindHajer.html', 'r') as f:
    htmlFile = f.read()
f.close()    

In [36]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(htmlFile, 'html.parser')

In [37]:
def findStalkers(soup, contentKey = "InitialChatFriendsList"):
    for tag in soup.find_all():
        stalkers = filter( lambda content: contentKey in content, tag.contents )
        if stalkers:
            return stalkers[0]

In [38]:
stalkerBlock = findStalkers(soup)

In [29]:
print type(stalkerBlock)

<class 'bs4.element.NavigableString'>


In [41]:
m = re.search('(?<=,list:)[.+?(?=])', stalkerBlock)

error: unbalanced parenthesis

In [40]:
stalkerIds = m.group().encode('ascii','ignore').replace('"',"").split(",")

AttributeError: 'NoneType' object has no attribute 'group'

In [34]:
import numpy as np

def getStalkerCreds(obj):
    idAndRank = obj.split("-")
    fId = idAndRank[0]
    
    rank = idAndRank[-1] if len(idAndRank) > 1 else ""
    return (fId,rank)    

In [18]:
stalkerIdAndRanks = map(getStalkerCreds, stalkerIds)

In [19]:
stalkerIds = map( lambda (fId, rank): fId, stalkerIdAndRanks )

In [20]:
stalkerRanks = map( lambda (fId, rank): rank, stalkerIdAndRanks )

In [21]:
import pandas as pd
stalkerDf = pd.DataFrame({'stalkerIds' : stalkerIds, 'stalkerRanks': stalkerRanks})

In [22]:
stalkerDf.head()

,stalkerIds,stalkerRanks
0,516881924,2
1,577585571,2
2,1113990472,2
3,592203656,2
4,507854573,2


In [23]:
stalkerCount = stalkerDf.groupby(["stalkerIds"]).apply(lambda group: len(group)).reset_index(name = "count")

In [24]:
finalStalkerDf = stalkerDf.merge(stalkerCount, how = "left", left_on = "stalkerIds", right_on = "stalkerIds")

In [25]:
finalStalkerDf.head()

,stalkerIds,stalkerRanks,count
0,516881924,2,2
1,577585571,2,2
2,1113990472,2,2
3,592203656,2,2
4,507854573,2,2


In [26]:
profiles = re.search('(?<=,shortProfiles:).+?(?=}})', stalkerBlock).group().encode('ascii','ignore')

In [608]:
print profiles

{"516881924":{id:"516881924",name:"Ali Alaraji",firstName:"Ali",vanity:"ali.alaraji.9",thumbSrc:"https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-1/p32x32/13690705_10153885277836925_6444860153419814908_n.jpg?oh=aa3caa1bc655dcb894d5bdbc68ae4302&oe=5914BDF8",uri:"https://www.facebook.com/ali.alaraji.9",gender:2,i18nGender:16777216,type:"friend",is_friend:true,mThumbSrcSmall:null,mThumbSrcLarge:null,dir:null,searchTokens:["Alaraji","Ali"],alternateName:"",is_nonfriend_messenger_contact:false},"577585571":{id:"577585571",name:"Khawaja Ahmad",firstName:"Khawaja",vanity:"khawajaahmad07",thumbSrc:"https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-1/p32x32/10291868_10153001112710572_6323637106102569664_n.jpg?oh=157dc8e88c2f9d16aa27f6362b76d260&oe=591EC584",uri:"https://www.facebook.com/khawajaahmad07",gender:2,i18nGender:16777216,type:"friend",is_friend:true,mThumbSrcSmall:null,mThumbSrcLarge:null,dir:null,searchTokens:["Ahmad","Khawaja"],alternateName:"",is_nonfriend_messenger_contact:false},"776360028"

In [27]:
profiles2 =  re.findall('(?<=:){.+?(?<=})', profiles)

In [28]:
print profiles2[6]

{id:"884655270",name:"Daniela Arredondo",firstName:"Daniela",vanity:"daniela.arredondo.779",thumbSrc:"https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-1/c8.0.32.32/p32x32/10475461_10154363643790271_2672356155483715337_n.jpg?oh=4e2617f522824d39b68093e5650e46cb&oe=59072A5D",uri:"https://www.facebook.com/daniela.arredondo.779",gender:1,i18nGender:33554432,type:"friend",is_friend:true,mThumbSrcSmall:null,mThumbSrcLarge:null,dir:null,searchTokens:["Arredondo","Daniela"],alternateName:"",is_nonfriend_messenger_contact:false}


In [29]:
def loadJsonStr(profile):
    profile = profile.encode('ascii','ignore')
    
    j = re.sub(r"{\s*'?(\w)", r'{"\1', profile)
    j = re.sub(r",\s*'?(\w)", r',"\1', j)
    j = re.sub(r"(\w)'?\s*:(?!/)", r'\1":', j)
    j = re.sub(r":\s*'(\w+)'\s*([,}])", r':"\1"\2', j) 
    j = re.sub(r",\s*]", "]", j)
    
    return json.loads(j)

In [30]:
profiles3 =  map(loadJsonStr, profiles2)
keys = profiles3[-1].keys()

In [31]:
dictForDf = dict( map( lambda keyName: (str(keyName), map(lambda x: x[str(keyName)], profiles3)), keys ) )

In [32]:
profilesDf = pd.DataFrame(dictForDf)

In [33]:
profilesDf[]

SyntaxError: invalid syntax (<ipython-input-33-f9ffa585af04>, line 1)

In [34]:
finalDF = finalStalkerDf.merge(profilesDf, how = "left", left_on = "stalkerIds", right_on = "id")

In [35]:
finalDF.drop(["dir", "mThumbSrcLarge","mThumbSrcSmall"], inplace = True, axis = 1)

In [36]:
finalDF.head()

,stalkerIds,stalkerRanks,count,alternateName,firstName,gender,i18nGender,id,is_friend,is_nonfriend_messenger_contact,name,searchTokens,thumbSrc,type,uri,vanity
0,516881924,2,2,,Ali,2.0,16777216.0,516881924,True,False,Ali Alaraji,"[Alaraji, Ali]",https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-1/...,friend,https://www.facebook.com/ali.alaraji.9,ali.alaraji.9
1,577585571,2,2,,Khawaja,2.0,16777216.0,577585571,True,False,Khawaja Ahmad,"[Ahmad, Khawaja]",https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-1/...,friend,https://www.facebook.com/khawajaahmad07,khawajaahmad07
2,1113990472,2,2,,Sabaoon,2.0,16777216.0,1113990472,True,False,Sabaoon Sangar,"[Sangar, Sabaoon]",https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-1/...,friend,https://www.facebook.com/sabaoon.sangar,sabaoon.sangar
3,592203656,2,2,,Amin,2.0,16777216.0,592203656,True,False,Amin Mohamoud,"[Mohamoud, Amin]",https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-1/...,friend,https://www.facebook.com/amin.mohamoud,amin.mohamoud
4,507854573,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [792]:
df = finalDF.replace(['',None], np.nan, axis = 0)

In [799]:
df.head()

,stalkerIds,stalkerRanks,count,alternateName,firstName,gender,i18nGender,id,is_friend,is_nonfriend_messenger_contact,name,searchTokens,thumbSrc,type,uri,vanity
0,516881924,2,2,NaN,Ali,2.0,16777216.0,516881924,True,False,Ali Alaraji,"[Alaraji, Ali]",https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-1/...,friend,https://www.facebook.com/ali.alaraji.9,ali.alaraji.9
1,577585571,2,2,NaN,Khawaja,2.0,16777216.0,577585571,True,False,Khawaja Ahmad,"[Ahmad, Khawaja]",https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-1/...,friend,https://www.facebook.com/khawajaahmad07,khawajaahmad07
2,1113990472,2,2,NaN,Sabaoon,2.0,16777216.0,1113990472,True,False,Sabaoon Sangar,"[Sangar, Sabaoon]",https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-1/...,friend,https://www.facebook.com/sabaoon.sangar,sabaoon.sangar
3,592203656,2,2,NaN,Amin,2.0,16777216.0,592203656,True,False,Amin Mohamoud,"[Mohamoud, Amin]",https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-1/...,friend,https://www.facebook.com/amin.mohamoud,amin.mohamoud
4,507854573,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [880]:
finalDF[finalDF.columns] = finalDF[finalDF.columns].astype(str)

In [864]:
finalDF.replace(['','NaN','nan', float('NaN')], np.nan, axis = 0, inplace = True)

In [872]:
df

,stalkerIds,stalkerRanks,count,alternateName,firstName,gender,i18nGender,id,is_friend,is_nonfriend_messenger_contact,name,searchTokens,thumbSrc,type,uri,vanity
0,516881924,2,2,nan,Ali,2.0,16777216.0,516881924,True,False,Ali Alaraji,"[u'Alaraji', u'Ali']",https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-1/...,friend,https://www.facebook.com/ali.alaraji.9,ali.alaraji.9
1,577585571,2,2,nan,Khawaja,2.0,16777216.0,577585571,True,False,Khawaja Ahmad,"[u'Ahmad', u'Khawaja']",https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-1/...,friend,https://www.facebook.com/khawajaahmad07,khawajaahmad07
2,1113990472,2,2,nan,Sabaoon,2.0,16777216.0,1113990472,True,False,Sabaoon Sangar,"[u'Sangar', u'Sabaoon']",https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-1/...,friend,https://www.facebook.com/sabaoon.sangar,sabaoon.sangar
3,592203656,2,2,nan,Amin,2.0,16777216.0,592203656,True,False,Amin Mohamoud,"[u'Mohamoud', u'Amin']",https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-1/...,friend,https://www.facebook.com/amin.mohamoud,amin.mohamoud
4,507854573,2,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5,776360028,2,2,nan,Yazan,2.0,16777216.0,776360028,True,False,Yazan Moghrabi,"[u'Moghrabi', u'Yazan']",https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-1/...,friend,https://www.facebook.com/yazan.moghrabi,yazan.moghrabi
6,504011056,2,2,nan,Ib,2.0,16777216.0,504011056,True,False,Ib Barbour,"[u'Barbour', u'Ib']",https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-1/...,friend,https://www.facebook.com/ibrahim.barbour.9,ibrahim.barbour.9
7,708290653,2,2,nan,Mouadh,2.0,16777216.0,708290653,True,False,Mouadh Xidouri,"[u'Xidouri', u'Mouadh']",https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-1/...,friend,https://www.facebook.com/R0ad.Runner,R0ad.Runner
8,100008029517033,2,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
9,100006729130380,2,2,nan,Nazar,2.0,16777216.0,100006729130380,True,False,Nazar Moh'd,"[u""Moh'd"", u'Nazar']",https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-1/...,friend,https://www.facebook.com/nazar.mohamad.92,nazar.mohamad.92


In [875]:
finalDF.replace(['NaN','nan', float('NaN')], np.nan, axis = 0, inplace = True)

In [881]:
finalDF[ finalDF.name == 'nan' ]

,stalkerIds,stalkerRanks,count,alternateName,firstName,gender,i18nGender,id,is_friend,is_nonfriend_messenger_contact,name,searchTokens,thumbSrc,type,uri,vanity
4,507854573,2,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
8,100008029517033,2,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
11,100005942104843,2,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
14,100007816008571,2,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
16,100009495768692,2,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
17,648156327,2,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
18,100007916201700,2,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
19,1169653459,2,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
20,100000565000141,2,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
22,754184733,2,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [66]:
def findMissingName(obj):
    name = obj[1]
    
    if name:
        profileId = obj[0]
        html = urlopen("https://www.facebook.com/{}".format(profileId)).read()
        
        soup = BeautifulSoup(html, 'html.parser')
        nameTag = findStalkers(soup, "name")
        name = re.search('(?<="name":).+?(?=,)', nameTag)
        return str(name.group()).replace('"',"")
    
    return name   

In [67]:
from urllib2 import urlopen

In [70]:
findMissingName(pd.Series(["521025612","nan"]))

'Jin W Kim'

In [ ]:
html = urlopen("https://www.facebook.com/516881924").read()

In [ ]:
print html

In [ ]:
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
nameTag = findStalkers(soup, "name")

In [ ]:
m = re.search('(?<="name":).+?(?=",)', nameTag)

In [ ]:
print type(str(m.group()))

In [ ]:
if finalDF["dir"].tolist()[4]:
    print "hello"

In [ ]:
df = finalDF.replace(, np.nan, axis = 0)
df.head()

In [40]:
df[ df['dir'] == np.nan ] 

NameError: name 'df' is not defined

In [41]:
finalDF[finalDF.isnull()] = np.nan

In [42]:
finalDF[ df['dir'] == np.nan ]

NameError: name 'df' is not defined

In [43]:
finalDF["someTest"] = finalDF[['stalkerIds',"count"]].apply( lambda row: row[0] + row[1], axis = 1)

TypeError: ("cannot concatenate 'str' and 'int' objects", u'occurred at index 0')

In [44]:
finalDF.someTest.head()

AttributeError: 'DataFrame' object has no attribute 'someTest'